In [2]:
import pandas as pd

import pandas as pd 
import openai
import json
from collections import Counter
import random
import time
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from src.utils import *

In [3]:
import os
print("Current Directory:", os.getcwd())

Current Directory: c:\Users\RodrigoMartínezAlons\OneDrive - Sparrow Networks GmbH\Python\categorization\LLMs


### 1) Data. Extract the data is the desired format

In [7]:
df=pd.read_excel('data/dict/dict_english.xlsx')

In [4]:
# df = pd.read_excel('final_dict.xlsx', sheet_name='Categories')  # Replace 'SheetName' with the name of the sheet you want

In [8]:
df.head()

,Categories,No.,description,Classification Examples,Examples Continued,Sparrow Description,Comment JS
0,Facility consumables,99501430,"Operational Supplies, PPE, laboratory Supplies...",Adhesives,NaN,NaN,NaN
1,NaN,NaN,NaN,Cleaning Products,NaN,NaN,NaN
2,NaN,NaN,NaN,Clothing,NaN,NaN,NaN
3,NaN,NaN,NaN,Compressed Air,NaN,NaN,NaN
4,NaN,NaN,NaN,Container Cylinders,NaN,NaN,NaN


In [9]:
df.shape

(134, 7)

Disctionary with all terms inside the categories

## General Terms

### English

In [122]:
categories = {"Facility Consumables": df['Classification Examples'].loc[0:16].tolist(),
    "Fasteners": df['Classification Examples'].loc[17:34].tolist(),
    "Valves, Actuator, Fittings": df['Classification Examples'].loc[35:49].tolist(),
    "Equipment OEM Spare Parts": df['Classification Examples'].loc[50:51].tolist(),
    "Electrical Installation Materials": df['Classification Examples'].loc[52:68].tolist(),
    "Piping Materials": df['Classification Examples'].loc[69:84].tolist(),
    "Gas, Water and Sewage Installation": df['Classification Examples'].loc[85:101].tolist(),
    "Measurement technology, Process measurement technology": df['Classification Examples'].loc[102:112].tolist(),
    # "Unclassified ": df['Classification Examples'].loc[113:123].tolist(),
    "Drives ": df['Classification Examples'].loc[124:134].tolist(),}

categories_data = {category: [item for item in items if pd.notna(item)] for category, items in categories.items()}
categories_data

{'Facility Consumables': ['Adhesives',
  'Cleaning Products',
  'Clothing',
  'Compressed Air',
  'Container Cylinders',
  'Gas Canisters',
  'Greases',
  'Keys - Security (Physical), Lock and Key',
  'Lubricants',
  'Oils',
  'Out of scope items',
  'Paints',
  'PPE',
  'Safety Equipment',
  'Software and Licenses',
  'Tools and Tooling',
  'Varnish'],
 'Fasteners': ['Ballscrews',
  'Bearings',
  'Bolts',
  'Cam Followers',
  'Cam Rings',
  'Clamps',
  'Connectors - Non Electrical, non piping',
  'Conveyor Belts',
  'Gaskets',
  'Joints - non piping',
  'Keys - Kewyays, shaft holding, fits',
  'Levers',
  'Mechanical Seals',
  'Mounting Kits',
  'Nuts',
  'O Rings',
  'O Ring Kits',
  'Plugs - Non Electrical'],
 'Valves, Actuator, Fittings': ['Actuators',
  'Actuator Pipes',
  'Dampers',
  'Diffusers',
  'Guide Units',
  'Junction Boxes',
  'Membranes',
  'Pneumatic Cylinders',
  'Repair Kits - Cylinders, Valves',
  'Restrictors',
  'Seats',
  'Shock Absorbers',
  'Terminals - Airline

Eliminate out of scope from the data

In [166]:
# Example usage
data = {"Category1": ["word1", "word2", "word3"], "Category2": ["word4", "word5"]}
updated_data = modify_category(data, "Category2", "Category1", ["word1"], ["word6"])
print(updated_data)

Updated Dictionary:
{'Facility Consumables': ['Adhesives', 'Cleaning Products', 'Clothing', 'Compressed Air', 'Container Cylinders', 'Gas Canisters', 'Greases', 'Keys - Security (Physical), Lock and Key', 'Lubricants', 'Oils', 'Paints', 'PPE', 'Safety Equipment', 'Software and Licenses', 'Tools and Tooling', 'Varnish'], 'Fasteners': ['Ballscrews', 'Bearings', 'Bolts', 'Cam Followers', 'Cam Rings', 'Clamps', 'Connectors - Non Electrical, non piping', 'Conveyor Belts', 'Gaskets', 'Joints - non piping', 'Keys - Kewyays, shaft holding, fits', 'Levers', 'Mechanical Seals', 'Mounting Kits', 'Nuts', 'O Rings', 'O Ring Kits', 'Plugs - Non Electrical'], 'Valves, Actuator, Fittings': ['Actuators', 'Actuator Pipes', 'Dampers', 'Diffusers', 'Guide Units', 'Junction Boxes', 'Membranes', 'Pneumatic Cylinders', 'Repair Kits - Cylinders, Valves', 'Restrictors', 'Seats', 'Shock Absorbers', 'Terminals - Airlines', 'Vacuum Ejectors', 'Valves'], 'Equipment OEM Spare Parts': ['Abstreifer', 'Spindles'], 'El

Create the dictionary

In [125]:
# Function to create a dictionary mapping terms to categories
def map_terms_to_categories(categories_data):
    term_to_category = {}
    for category, terms in categories_data.items():
        for term in terms:
            term_lower = term.lower()  # Normalize terms to lowercase
            term_to_category[term_lower] = category
    return term_to_category

# Generate the dictionary
dict = map_terms_to_categories(updated_data)
dict

{'adhesives': 'Facility Consumables',
 'cleaning products': 'Facility Consumables',
 'clothing': 'Facility Consumables',
 'compressed air': 'Facility Consumables',
 'container cylinders': 'Facility Consumables',
 'gas canisters': 'Facility Consumables',
 'greases': 'Facility Consumables',
 'keys - security (physical), lock and key': 'Facility Consumables',
 'lubricants': 'Facility Consumables',
 'oils': 'Facility Consumables',
 'paints': 'Facility Consumables',
 'ppe': 'Facility Consumables',
 'safety equipment': 'Facility Consumables',
 'software and licenses': 'Facility Consumables',
 'tools and tooling': 'Facility Consumables',
 'varnish': 'Facility Consumables',
 'ballscrews': 'Fasteners',
 'bearings': 'Fasteners',
 'bolts': 'Fasteners',
 'cam followers': 'Fasteners',
 'cam rings': 'Fasteners',
 'clamps': 'Fasteners',
 'connectors - non electrical, non piping': 'Fasteners',
 'conveyor belts': 'Fasteners',
 'gaskets': 'Fasteners',
 'joints - non piping': 'Fasteners',
 'keys - kewyay

In [126]:
unique_categories = set(dict.values())
num_categories = len(unique_categories)
print(f"Number of unique categories: {num_categories}")
print(f"Categories: {unique_categories}")

Number of unique categories: 9
Categories: {'Valves, Actuator, Fittings', 'Piping Materials', 'Measurement technology, Process measurement technology', 'Fasteners', 'Drives ', 'Gas, Water and Sewage Installation', 'Equipment OEM Spare Parts', 'Electrical Installation Materials', 'Facility Consumables'}


In [127]:
value_counts = Counter(dict.values())
repeated_values = {key: count for key, count in value_counts.items() if count > 1}
value_counts

Counter({'Fasteners': 18,
         'Electrical Installation Materials': 17,
         'Gas, Water and Sewage Installation': 17,
         'Facility Consumables': 16,
         'Piping Materials': 16,
         'Valves, Actuator, Fittings': 15,
         'Measurement technology, Process measurement technology': 11,
         'Drives ': 10,
         'Equipment OEM Spare Parts': 2})

In [160]:
dict

{'adhesives': 'Facility Consumables',
 'cleaning products': 'Facility Consumables',
 'clothing': 'Facility Consumables',
 'compressed air': 'Facility Consumables',
 'container cylinders': 'Facility Consumables',
 'gas canisters': 'Facility Consumables',
 'greases': 'Facility Consumables',
 'keys - security (physical), lock and key': 'Facility Consumables',
 'lubricants': 'Facility Consumables',
 'oils': 'Facility Consumables',
 'paints': 'Facility Consumables',
 'ppe': 'Facility Consumables',
 'safety equipment': 'Facility Consumables',
 'software and licenses': 'Facility Consumables',
 'tools and tooling': 'Facility Consumables',
 'varnish': 'Facility Consumables',
 'ballscrews': 'Fasteners',
 'bearings': 'Fasteners',
 'bolts': 'Fasteners',
 'cam followers': 'Fasteners',
 'cam rings': 'Fasteners',
 'clamps': 'Fasteners',
 'connectors - non electrical, non piping': 'Fasteners',
 'conveyor belts': 'Fasteners',
 'gaskets': 'Fasteners',
 'joints - non piping': 'Fasteners',
 'keys - kewyay

General terms in German

In [164]:
from deep_translator import GoogleTranslator

In [165]:
# Translate the keys to German
translated_categories = {GoogleTranslator(source='auto', target='de').translate(key): value for key, value in dict.items()}
translated_categories

{'Klebstoffe': 'Facility Consumables',
 'Reinigungsmittel': 'Facility Consumables',
 'Kleidung': 'Facility Consumables',
 'Druckluft': 'Facility Consumables',
 'Behälterzylinder': 'Facility Consumables',
 'Gaskartuschen': 'Facility Consumables',
 'Fette': 'Facility Consumables',
 'Schlüssel - Sicherheit (physisch), Schloss und Schlüssel': 'Facility Consumables',
 'Schmierstoffe': 'Facility Consumables',
 'Öle': 'Facility Consumables',
 'Farben': 'Facility Consumables',
 'psa': 'Facility Consumables',
 'Sicherheitsausrüstung': 'Facility Consumables',
 'Software und Lizenzen': 'Facility Consumables',
 'Werkzeuge und Werkzeugausstattung': 'Facility Consumables',
 'Lack': 'Facility Consumables',
 'Kugelumlaufspindeln': 'Fasteners',
 'Lager': 'Fasteners',
 'Bolzen': 'Fasteners',
 'Nockenstößel': 'Fasteners',
 'Nockenringe': 'Fasteners',
 'Klemmen': 'Fasteners',
 'Verbindungselemente - nicht elektrisch, nicht rohrförmig': 'Fasteners',
 'Förderbänder': 'Fasteners',
 'Dichtungen': 'Fasteners',

### Spanish

## Specific Terms

### English

In [1]:
import pandas as pd

In [1]:
df_specific = pd.read_excel('final_dict.xlsx', sheet_name='EN Dictionary')  # Replace 'SheetName' with the name of the sheet you want

NameError: name 'pd' is not defined

In [15]:
df_specific.group_label.value_counts()

group_label
Electrical Installation Materials, device                  679
Fasteners                                                  288
Facility consumables                                       164
Valves, Actuator, Fittings                                 144
Measurement technology, Process measurement technology     132
Gas, water and sewage installation                          91
Drives                                                      85
Unclassified                                                56
Piping Materials                                            53
Equipment OEM Spare Parts                                   16
Name: count, dtype: int64

Take only the ones flagged True

In [16]:
df_specific_nouns=df_specific[df_specific['code_flag']]
df_specific_nouns = df_specific_nouns[['noun_en', 'group_label']]
df_specific_nouns = df_specific_nouns[df_specific_nouns['group_label'] != "Unclassified"]
df_specific_nouns.shape

(1319, 2)

In [146]:
df_specific_nouns.group_label.value_counts()

group_label
Electrical Installation Materials, device                  733
Fasteners                                                  435
Valves, Actuator, Fittings                                 261
Gas, water and sewage installation                         162
Measurement technology, Process measurement technology     161
Facility consumables                                       142
Piping Materials                                            92
Drives                                                      72
Equipment OEM Spare Parts                                   62
Name: count, dtype: int64

Create the dictionaries

In [147]:
result = df_specific_nouns.groupby("group_label")["noun_de"].apply(list).to_dict()
dict_specific = map_terms_to_categories(result)
dict_specific

{'wechselstrommotoren': 'Drives',
 'nockenring': 'Drives',
 'motorreductor': 'Drives',
 'motorreduktor': 'Drives',
 'drehstrommotor': 'Drives',
 'motorbaugruppe': 'Drives',
 'motor modul': 'Drives',
 'schraubenmotor': 'Drives',
 'schrittmotor': 'Drives',
 'schrittmotormodul': 'Drives',
 'zahnrad': 'Drives',
 'getriebesteckwelle': 'Drives',
 'türantrieb': 'Drives',
 'achse': 'Drives',
 'antriebswelle': 'Drives',
 'getriebemotor': 'Drives',
 'antriebsstange': 'Drives',
 'antriebsriemen': 'Drives',
 'ausgleichkupplung': 'Drives',
 'ausgleichskupplung': 'Drives',
 'einzelmotor-modul': 'Drives',
 'flachriemen': 'Drives',
 'flexo kupplung': 'Drives',
 'flexokupplung': 'Drives',
 'getriebe': 'Drives',
 'getriebeübersetzung': 'Drives',
 'gliederkeilriemen': 'Drives',
 'hochleistungsflachriemen': 'Drives',
 'kegelradgetriebe': 'Drives',
 'kegelstirnradgetriebe': 'Drives',
 'keilriemen': 'Drives',
 'keilriemenrad': 'Drives',
 'keilrippenriemen': 'Drives',
 'keilrscheibe': 'Drives',
 'kette': 'Dr

In [148]:
with open('dict_ft/dict_de.json', 'w') as json_file:
    json.dump(dict_specific, json_file, indent=4)

In [151]:
dict_specific

{'wechselstrommotoren': 'Drives',
 'nockenring': 'Drives',
 'motorreductor': 'Drives',
 'motorreduktor': 'Drives',
 'drehstrommotor': 'Drives',
 'motorbaugruppe': 'Drives',
 'motor modul': 'Drives',
 'schraubenmotor': 'Drives',
 'schrittmotor': 'Drives',
 'schrittmotormodul': 'Drives',
 'zahnrad': 'Drives',
 'getriebesteckwelle': 'Drives',
 'türantrieb': 'Drives',
 'achse': 'Drives',
 'antriebswelle': 'Drives',
 'getriebemotor': 'Drives',
 'antriebsstange': 'Drives',
 'antriebsriemen': 'Drives',
 'ausgleichkupplung': 'Drives',
 'ausgleichskupplung': 'Drives',
 'einzelmotor-modul': 'Drives',
 'flachriemen': 'Drives',
 'flexo kupplung': 'Drives',
 'flexokupplung': 'Drives',
 'getriebe': 'Drives',
 'getriebeübersetzung': 'Drives',
 'gliederkeilriemen': 'Drives',
 'hochleistungsflachriemen': 'Drives',
 'kegelradgetriebe': 'Drives',
 'kegelstirnradgetriebe': 'Drives',
 'keilriemen': 'Drives',
 'keilriemenrad': 'Drives',
 'keilrippenriemen': 'Drives',
 'keilrscheibe': 'Drives',
 'kette': 'Dr

Mix all 3 specific dictionaries and see the result. Probably strongest approach we have

In [173]:
with open('dict_ft/dict_de.json', 'r') as f:
    dict_de = json.load(f)

with open('dict_ft/dict_en.json', 'r') as f:
    dict_en = json.load(f)

with open('dict_ft/dict_es.json', 'r') as f:
    dict_es = json.load(f)

dict_total = dict_de.copy()  # Make a copy to avoid modifying the original
dict_total.update(dict_en)
dict_total.update(dict_es)

In [175]:
dict_total

{'wechselstrommotoren': 'Drives',
 'nockenring': 'Drives',
 'motorreductor': 'Drives',
 'motorreduktor': 'Drives',
 'drehstrommotor': 'Drives',
 'motorbaugruppe': 'Drives',
 'motor modul': 'Drives',
 'schraubenmotor': 'Drives',
 'schrittmotor': 'Drives',
 'schrittmotormodul': 'Drives',
 'zahnrad': 'Drives',
 'getriebesteckwelle': 'Drives',
 'türantrieb': 'Drives',
 'achse': 'Drives',
 'antriebswelle': 'Drives',
 'getriebemotor': 'Drives',
 'antriebsstange': 'Drives',
 'antriebsriemen': 'Drives',
 'ausgleichkupplung': 'Drives',
 'ausgleichskupplung': 'Drives',
 'einzelmotor-modul': 'Drives',
 'flachriemen': 'Drives',
 'flexo kupplung': 'Drives',
 'flexokupplung': 'Drives',
 'getriebe': 'Drives',
 'getriebeübersetzung': 'Drives',
 'gliederkeilriemen': 'Drives',
 'hochleistungsflachriemen': 'Drives',
 'kegelradgetriebe': 'Drives',
 'kegelstirnradgetriebe': 'Drives',
 'keilriemen': 'Drives',
 'keilriemenrad': 'Drives',
 'keilrippenriemen': 'Drives',
 'keilrscheibe': 'Drives',
 'kette': 'Dr

### Spanish

### 2) Model

Split the data in training and test

In [176]:
# Split data into training (80%) and test (20%)
keys = list(dict_total.keys())
random.shuffle(keys)
split_index = int(len(keys) * 0.9)

training_keys = keys[:split_index]
test_keys = keys[split_index:]

In [177]:
# Create training and test datasets
training_data = transform_to_messages_format(training_keys, dict_total)
test_data = transform_to_messages_format(test_keys, dict_total)

# Save the datasets as JSONL files
with open("data/dict/training_data.jsonl", "w") as train_file:
    for entry in training_data:
        train_file.write(json.dumps(entry) + "\n")

with open("data/dict/test_data.jsonl", "w") as test_file:
    for entry in test_data:
        test_file.write(json.dumps(entry) + "\n")

# Display a sample of the training and test data
print("Training Data Sample:", training_data[:2])
print("Test Data Sample:", test_data[:2])

Training Data Sample: [{'messages': [{'role': 'system', 'content': 'You are a helpful assistant for product classification.'}, {'role': 'user', 'content': 'getriebesteckwelle'}, {'role': 'assistant', 'content': 'Drives'}]}, {'messages': [{'role': 'system', 'content': 'You are a helpful assistant for product classification.'}, {'role': 'user', 'content': 'parallelgreifer'}, {'role': 'assistant', 'content': 'Equipment OEM Spare Parts'}]}]
Test Data Sample: [{'messages': [{'role': 'system', 'content': 'You are a helpful assistant for product classification.'}, {'role': 'user', 'content': 'lagergehäuse'}, {'role': 'assistant', 'content': 'Fasteners'}]}, {'messages': [{'role': 'system', 'content': 'You are a helpful assistant for product classification.'}, {'role': 'user', 'content': 'cd rom'}, {'role': 'assistant', 'content': 'Electrical Installation Materials, device'}]}]


In [178]:
# Set the API key

with open("data/dict/training_data.jsonl", "rb") as train_file:
    train_response = openai.File.create(file=train_file, purpose='fine-tune')

with open("data/dict/test_data.jsonl", "rb") as valid_file:
    test_response = openai.File.create(file=valid_file, purpose='fine-tune')

In [179]:
# train_response = openai.File.create(file=training_data, purpose='fine-tune')
# test_response = openai.File.create(file=test_data, purpose='fine-tune')

train_file_id = train_response["id"]
valid_file_id = test_response["id"]

fine_tune_response = openai.FineTuningJob.create(
    training_file=train_file_id,
    validation_file=valid_file_id,
    model="gpt-4o-mini-2024-07-18",
    hyperparameters={
        "n_epochs": 3  # Only specify allowed hyperparameters like `n_epochs`
}
)


Changing hyperparameters to prevent possible overfitting. Also with balanced classes

In [ ]:
# # Retrieve file IDs
# train_file_id = train_response["id"]
# valid_file_id = test_response["id"]

# fine_tune_response = openai.FineTuningJob.create(
#     training_file=train_file_id,
#     validation_file=valid_file_id,
#     model="gpt-3.5-turbo",
#     hyperparameters={
#         "n_epochs": 5,                
#         "learning_rate_multiplier": 0.05,  
#         "batch_size": 8               
#     }
# )


In [ ]:
# job_id = "ftjob-OOodGI5t6FfH8xFfkwbhUsyI"
# job_status = openai.FineTuningJob.retrieve(id=job_id)
# print(job_status)

{
  "object": "fine_tuning.job",
  "id": "ftjob-OOodGI5t6FfH8xFfkwbhUsyI",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1736423075,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-MXa32HRO92uJDJRTrzMzCZYo",
  "result_files": [],
  "status": "running",
  "validation_file": "file-VTR3gnuDCFdEUrfFbiqvwg",
  "training_file": "file-D6AMGWdNzP7xzuDKNbrHFu",
  "hyperparameters": {
    "n_epochs": 4,
    "batch_size": 5,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": null,
  "seed": 1622940221,
  "estimated_finish": null,
  "integrations": [],
  "method": {
    "type": "supervised",
    "supervised": {
      "hyperparameters": {
        "n_epochs": 4,
        "batch_size": 5,
        "learning_rate_multiplier": 2.0
      }
    }
  }
}


In [ ]:
print(fine_tune_response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-OOodGI5t6FfH8xFfkwbhUsyI",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1736423075,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-MXa32HRO92uJDJRTrzMzCZYo",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-VTR3gnuDCFdEUrfFbiqvwg",
  "training_file": "file-D6AMGWdNzP7xzuDKNbrHFu",
  "hyperparameters": {
    "n_epochs": 4,
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": null,
  "seed": 1622940221,
  "estimated_finish": null,
  "integrations": [],
  "method": {
    "type": "supervised",
    "supervised": {
      "hyperparameters": {
        "batch_size": "auto",
        "learning_rate_multiplier": "auto",
        "n_epochs": 4
      }
    }
  }
}


** Decoding for some words with different formats or characters. Still gpt models do understand them

In [ ]:
# Example encoded word
encoded_word = "ber\u00fchrungssensor"

# Decode Unicode
decoded_word = encoded_word.encode('utf-8').decode('unicode_escape')
print(decoded_word)  # Output: berührungssensor

berÃ¼hrungssensor


In [97]:
# Use the fine-tuned model for predictions
response = openai.ChatCompletion.create(
    model="ft:gpt-4o-mini-2024-07-18:sparrow::ArmpLSLD",  # Replace with your model ID
    messages=[
        {"role": "system", "content": "You are a helpful assistant for product classification.You classify products."
                 "You can only use the same categories that are in the training data. You will have short descriptions as inputs so you can have more context on every part you have to categorize"},
        {"role": "user", "content": "Machine Distributor A3/1 X098"}])

# Print the prediction
print(response['choices'][0]['message']['content'])

Facility Consumables
